In [ ]:
from scipy.io import loadmat
import numpy as np
import graph_tool as gt
from graph_tool import inference
from pytictoc import TicToc
from joblib import Parallel, delayed
from readFile import *
from SBMs import *
import config

#set global variables from config
choose_run = config.choose_run
iterations = config.iterations
directory = config.loc
runs = config.runs
subject_list = config.subject_list

#set up function for parallelization
def obtain_nested_partition(i, iterations):
    
    adj = loadmat(subject_list[i])
    PC = runs[i]
    adj = adj['network_adjacency'][0][PC][0][0]['adjacency'][0]['adjacency'][0]
    partition = nested_SBM(adj, iterations)

    return partition

#initialize timing
t = TicToc()
t.tic()

#set range for desired indices within full dataset
start_index = 0
end_index = 3

#set number of available cores for parallelization
cores = 3

#parallelize according to number of available cores - verbose for job updates
results = Parallel(n_jobs=cores, verbose = 30, timeout = 30000)(delayed(obtain_nested_partition)(i, iterations) for i in range(start_index,end_index))

#end timing
t.toc()

#choose file name
file_name = "example_nested.csv"

#save results as csv - each row within csv file corresponds to a different subject and each column (of 1088) is the node of corresponding index
np.savetxt(file_name, results, delimiter=',')

Data directory:/home/daoutidi/shared/new_adj/
Current run: min_med_conn_run
Number of iterations set to 10
Number of files in file list: 5955
Number of subjects: 5251


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 tasks      | elapsed:  3.2min
